# Load Data

In [102]:
import numpy as np
import glob
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from sklearn.metrics import accuracy_score
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, TensorDataset
from imblearn.over_sampling import SMOTE
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score

In [103]:
birds_labels = {
    "other": 0,
    "comcuc": 1,
    "cowpig1": 2,
    "eucdov": 3,
    "eueowl1": 4,
    "grswoo": 5,
    "tawowl1": 6
}

birds = [bird for bird in list(birds_labels.keys()) if bird != "other"]

In [104]:
def unique_rows(matrix):
    indexes = []
    labels = []
    
    for i, row in enumerate(matrix):
        if np.unique(row).size == 1:
            indexes.append(i)
            labels.append(np.unique(row)[0])
    
    indexes = np.array(indexes)
    labels = np.array(labels)
    
    return indexes, labels

In [105]:
def load_data(bird):
    labels = []
    features = []
    bird_id = birds_labels[bird]
    
    path = f'./data/{bird}/'
    labels_files = glob.glob(path + '*labels.npy')
    counter = None
    
    for i, file in enumerate(labels_files):
        print(f'{bird}: {i + 1}/{len(labels_files)}', end='\r')
        counter = i
        data_id = path + ''.join(file.split(".labels.npy")).split('/')[-1] + '.npy'
        
        annotations = np.load(file)
        feature = np.load(data_id)
        
        ind, label = unique_rows(annotations)
        
        if len(ind) == 0:
            continue
        
        labels.append(label)
        features.append(feature[ind])

    print('\n')
    labels = np.concatenate(labels)
    features = np.concatenate(features)
    
    return labels, features

In [106]:
X = []
y = []

for bird in birds:
    labels, features = load_data(bird)
    X.append(features)
    y.append(labels)
    
X = np.concatenate(X)
y = np.concatenate(y)

comcuc: 200/200

cowpig1: 200/200

eucdov: 200/200

eueowl1: 200/200

grswoo: 200/200

tawowl1: 200/200



# Data Normalization

In [107]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Balancing the Dataset

In [108]:
# Assuming X is your feature matrix and Y are the labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42) # You can adjust the strategy as per your requirement
X_train, y_train = smote.fit_resample(X_train, y_train)

# Neural Network

In [109]:
class BirdClassifier(nn.Module):
    def __init__(self):
        super(BirdClassifier, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(17408, 1024)  # Updated input dimension
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 7)  # 7 output classes

    def forward(self, x):
        x = x.unsqueeze(1)  # (N, 548) -> (N, 1, 548)
        x = self.pool(F.relu(self.conv1(x)))  # conv1 + ReLU + max pooling
        x = self.pool(F.relu(self.conv2(x)))  # conv2 + ReLU + max pooling
        x = self.pool(F.relu(self.conv3(x)))  # conv3 + ReLU + max pooling
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))  # fc1 + ReLU
        x = F.relu(self.fc2(x))  # fc2 + ReLU
        x = F.softmax(self.fc3(x), dim=1)  # fc3 + softmax
        return x


# class AudioCNN(nn.Module):
#     def __init__(self):
#         super(AudioCNN, self).__init__()
#         self.conv1 = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm1d(64)
#         self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
#         self.bn2 = nn.BatchNorm1d(128)
#         self.conv3 = nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1)
#         self.bn3 = nn.BatchNorm1d(256)
#         self.conv4 = nn.Conv1d(256, 512, kernel_size=3, stride=1, padding=1)
#         self.bn4 = nn.BatchNorm1d(512)
#         self.conv5 = nn.Conv1d(512, 1024, kernel_size=3, stride=1, padding=1)
#         self.bn5 = nn.BatchNorm1d(1024)
#         self.pool = nn.MaxPool1d(2, 2)
#         self.dropout = nn.Dropout(0.5)
#         self.fc1 = nn.Linear(17408, 4096)  # Updated input dimension
#         self.fc2 = nn.Linear(4096, 2048)
#         self.fc3 = nn.Linear(2048, 1024)
#         self.fc4 = nn.Linear(1024, 512)
#         self.fc5 = nn.Linear(512, 7)  # 7 output classes

#     def forward(self, x):
#         x = x.unsqueeze(1)  # (N, 548) -> (N, 1, 548)
#         x = self.pool(F.relu(self.bn1(self.conv1(x))))  # conv1 + ReLU + BN + max pooling
#         x = self.pool(F.relu(self.bn2(self.conv2(x))))  # conv2 + ReLU + BN + max pooling
#         x = self.pool(F.relu(self.bn3(self.conv3(x))))  # conv3 + ReLU + BN + max pooling
#         x = self.pool(F.relu(self.bn4(self.conv4(x))))  # conv4 + ReLU + BN + max pooling
#         x = self.pool(F.relu(self.bn5(self.conv5(x))))  # conv5 + ReLU + BN + max pooling
#         x = x.view(x.size(0), -1)
#         x = self.dropout(F.relu(self.fc1(x)))  # fc1 + ReLU + Dropout
#         x = self.dropout(F.relu(self.fc2(x)))  # fc2 + ReLU + Dropout
#         x = self.dropout(F.relu(self.fc3(x)))  # fc3 + ReLU + Dropout
#         x = self.dropout(F.relu(self.fc4(x)))  # fc4 + ReLU + Dropout
#         x = self.fc5(x)  # fc5
#         return x

### Hyperparameters

In [110]:
# Define the hyperparameters
learning_rate = 1e-5
batch_size = 64
num_epochs = 100

from sklearn.utils.class_weight import compute_class_weight

# Assuming y is your vector of labels
classes = np.unique(y)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
class_weights = torch.tensor(weights, dtype=torch.float)

### Training

In [111]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)


# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).long().to(device)
y_test = torch.from_numpy(y_test).long().to(device)

# Create the DataLoader for training and test sets
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
model = AudioCNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

best_f1 = 0.0
best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
losses = []
f1_scores = []
accuracies = []

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Evaluate the model
    model.eval()  # Set the model to evaluation mode
    true_labels = []
    predicted_labels = []
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    epoch_acc = correct / total
    epoch_f1 = f1_score(true_labels, predicted_labels, average='macro')

    # deep copy the model
    if epoch_f1 > best_f1:
        best_f1 = epoch_f1
        best_model_wts = copy.deepcopy(model.state_dict())

    losses.append(epoch_loss)
    f1_scores.append(epoch_f1)
    accuracies.append(epoch_acc)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.5f}, Accuracy: {epoch_acc:.5f}, F1-score: {epoch_f1:.5f}')

print('Best Accuracy: {:5f}, Best F1-score: {:5f}'.format(best_acc, best_f1))

# load best model weights
model.load_state_dict(best_model_wts)


mps
Epoch 1/100, Loss: 1.32189, Accuracy: 0.14789, F1-score: 0.17949
Epoch 2/100, Loss: 1.24752, Accuracy: 0.75977, F1-score: 0.50692
Epoch 3/100, Loss: 1.21645, Accuracy: 0.81918, F1-score: 0.66089
Epoch 4/100, Loss: 1.17924, Accuracy: 0.87240, F1-score: 0.74129
Epoch 5/100, Loss: 1.17334, Accuracy: 0.88076, F1-score: 0.76564
Epoch 6/100, Loss: 1.17061, Accuracy: 0.90982, F1-score: 0.81407
Epoch 7/100, Loss: 1.16881, Accuracy: 0.91088, F1-score: 0.81199
Epoch 8/100, Loss: 1.16785, Accuracy: 0.92269, F1-score: 0.83457
Epoch 9/100, Loss: 1.16739, Accuracy: 0.92205, F1-score: 0.84377
Epoch 10/100, Loss: 1.16686, Accuracy: 0.92865, F1-score: 0.85280
Epoch 11/100, Loss: 1.16650, Accuracy: 0.94754, F1-score: 0.87743
Epoch 12/100, Loss: 1.16635, Accuracy: 0.93532, F1-score: 0.85462
Epoch 13/100, Loss: 1.16615, Accuracy: 0.95614, F1-score: 0.89625
Epoch 14/100, Loss: 1.16605, Accuracy: 0.95819, F1-score: 0.89748
Epoch 15/100, Loss: 1.16605, Accuracy: 0.96070, F1-score: 0.90254
Epoch 16/100, L

<All keys matched successfully>

In [54]:
print('Best Accuracy: {:5f}, Best F1-score: {:5f}'.format(best_acc, best_f1))

# load best model weights
model.load_state_dict(best_model_wts)

Best Accuracy: 0.000000, Best F1-score: 0.959302


<All keys matched successfully>

# Load Test Data

In [112]:
def load_data():
    path = f'./python'
    files = glob.glob(path + '/test*.npy')
    files = sorted(files)
    
    result = []
    labels = [file.split('/')[-1].replace('.npy', '') for file in files]
    
    for i, file in enumerate(files):
        print(f"Loaded {i + 1}/{len(files)} test files.", end="\r")
        data = np.load(file)
        result.append(data)
        
    result = np.array(result)
    
    return result, labels

In [113]:
test_data, labels = load_data()

In [114]:
test_data = np.array([scaler.fit_transform(arr) for arr in test_data])

In [120]:
# Assuming X_unlabeled is a numpy array
X_unlabeled = torch.from_numpy(test_data).float().to(device)

# Create a DataLoader for the unlabeled data
unlabeled_dataset = TensorDataset(X_unlabeled)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)


# Prediction function
def predict(model, unlabeled_loader):
    model.eval()  # Ensure the model is in evaluation mode
    result = []
    predictions = []

    with torch.no_grad():
        for sample in test_data:
            X_unlabeled = torch.from_numpy(sample).float().to(device)
            # Create a DataLoader for the unlabeled data
            unlabeled_dataset = TensorDataset(X_unlabeled)
            unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)
            
            for inputs in unlabeled_loader:
                inputs = inputs[0].to(device)
                outputs = model(inputs)  # Pass data through model
                _, predicted = torch.max(outputs, 1)  # Get the predicted classes
                predictions.extend(predicted.cpu().numpy().tolist())  # Add the predictions to our list
            
            result.append(predictions)
            predictions = []

    return np.array(result)

# Predict
predictions = predict(model, unlabeled_loader)
np.count_nonzero(predictions)

11784

In [117]:
sub = ""

for i, prediction in enumerate(predictions):
    y_pred = prediction.astype(str)
    
    y_str = ','.join(y_pred)
    result = f'{labels[i]},{y_str}\n'
    sub += result
    
sub

'test00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,3,0,0,0,0,1,1,1,1,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,0,0,0,1,1,1,1,0,0,0,0,5,0,0,5,0,0,0,5,0,0,5,5,0,0,5,0,0,5,0,0,5,0,0,0,5,0,0,5,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,1,1,1,1,0,0,0,1,1,1,1,0,0,0,0,1,0,1,2,0,0,2,1,1,1,1,2,0,2,2,1,0,0,5,0,0,5,0,0,5,0,0,5,5,0,5,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,5,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,

In [118]:
with open ('./submission_unbalanced_2.csv', 'w') as file:
    file.write(sub)